In [120]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import wfdb
from wfdb import processing
import os
from functools import partial
from utils import plot_signal_with_r_peaks, load_patient
from tqdm import tqdm
import torch.nn as nn
import pywt
from collections import Counter 
from scipy import signal as sig



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [121]:
import shutil
import os

In [5]:
def preprocess(ecg):
    
    signal = ecg.flatten()

    # Apply DWT
    wavelet = 'db4'
    level = 9
    coeffs = pywt.wavedec(signal, wavelet, level=level)

    # Remove baseline wandering
    coeffs[1:] = [pywt.threshold(i, np.std(i)/2) for i in coeffs[1:]]
    filtered_signal = pywt.waverec(coeffs, wavelet)

    # Apply lowpass filter
    nyquist = record.fs / 2
    cutoff = 40
    b, a = sig.butter(4, cutoff/nyquist, 'low')
    filtered_signal = sig.filtfilt(b, a, filtered_signal)
    return filtered_signal

In [116]:
data_dir = 'mit-bih-arrhythmia' 
dst_dir = 'filtered-mit-bih-arrhythmia'

if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

In [141]:
# copy of annotation files
for filename in os.listdir(data_dir):

    if filename.endswith('.atr'):

        src_file = os.path.join(data_dir, filename)
        dst_file = os.path.join(dst_dir, filename)
        shutil.copy(src_file, dst_file)

In [128]:
record.comments

['68 M 1960 2851 x1',
 'Digoxin, Hydrochlorthiazide, Inderal, KCl',
 'The PVCs are uniform and late-cycle.  This record was taken from the same',
 'analog tape as record 201.']

In [189]:
record.baseline

[1024, 1024]

In [235]:
patients = list(set([x.split('.')[0] for x in os.listdir(f'./{data_dir}/')]))

for i, pat_num in zip(range(len(patients)), patients):
    record = wfdb.rdrecord(f'{data_dir}/{pat_num}')
    
    ecg = np.asarray(record.p_signal[:,0], dtype=np.float64)

    # Load R peak annotations stored as Sample number
    annotation = wfdb.rdann(f'{data_dir}/{pat_num}', 'atr')
    filtered_signal = preprocess(ecg)
    a = record.p_signal[:, 0]
    record.p_signal[:, 0] = filtered_signal
    b = record.p_signal[:, 0]
    
#     print(record.p_signal)
    wfdb.wrsamp(f'{pat_num}', sig_name = record.sig_name, fs = record.fs, 
            p_signal = record.p_signal, d_signal = None, fmt=record.fmt, units = record.units, 
                comments=record.comments, write_dir = 'filtered-mit-bih-arrhythmia')


    record = wfdb.rdsamp(f'filtered-mit-bih-arrhythmia/{pat_num}')
#     break

202
219
109
116
212
200
221
220
232
121
106
102
205
230
217
103
117
114
118
231
214
201
234
124
122
113
111
101
108
203
107
210
208
115
119
222
223
213
215
233
207
112
104
123
100
228
209
105


In [ ]:
#     wfdb.wrsamp(f'{pat_num}', sig_name = record.sig_name, fs = record.fs, 
#             p_signal = record.p_signal, baseline = record.baseline, d_signal = None, fmt=record.fmt,
#                 adc_gain = record.adc_gain, units = record.units, 
#                 comments=record.comments, write_dir = 'filtered-mit-bih-arrhythmia')


# np.transpose(np.array([filtered_signal, record.p_signal[:, 1]]), (1, 0))
